In [40]:
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.neural_network import MLPRegressor
import pandas as pd
from statsmodels.formula.api import ols
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from statsmodels.stats.api import anova_lm

In [18]:
df = pd.read_csv("../data/curated/outlier_removed_data.csv")

In [19]:
df = df[['postcode', 'property_type', 'num_beds', 'num_baths', 'num_parking', 'school_duration', 'school_distance', 'park_duration', 'park_distance', 'shop_duration', 'shop_distance', 'weekly_rent']]

In [42]:
t = ColumnTransformer(transformers=[
    ('onehot', OneHotEncoder(), ['postcode', 'property_type', 'num_beds', 'num_baths', 'num_parking']),
    ('scale', StandardScaler(), ['school_duration', 'school_distance', 'park_duration', 'park_distance', 'shop_duration', 'shop_distance'])
], remainder='passthrough')  # Default is to drop untransformed columns

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0:-1], df.iloc[:,-1], test_size=0.33, random_state=42)
X_train_transformed = t.fit_transform(X_train)
X_test_transformed = t.fit_transform(X_test)


In [87]:
nn_model = MLPRegressor(max_iter=1000, activation='logistic',
                        n_iter_no_change=15, random_state=4).fit(X_train_transformed, y_train)


/Users/matthewrush/virtualenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [89]:
print(nn_model.score(X_train_transformed, y_train))
mean_absolute_error(nn_model.predict(
    X_train_transformed), y_train)

0.8058969916642279


50.22528547829401

In [90]:
lm = ols(
    "weekly_rent ~  C(postcode)+ C(property_type)+ C(num_beds)+ C(num_baths)+ C(num_parking) + school_duration+ school_distance + park_duration+ park_distance+ shop_duration+ shop_distance", data=df).fit()
anova_lm(lm)


,df,sum_sq,mean_sq,F,PR(>F)
C(postcode),412.0,1.384783e+08,3.361123e+05,21.686364,0.000000e+00
C(property_type),13.0,6.666090e+07,5.127761e+06,330.849271,0.000000e+00
C(num_beds),10.0,8.753932e+07,8.753932e+06,564.814154,0.000000e+00
C(num_baths),5.0,2.496466e+07,4.992931e+06,322.149875,0.000000e+00
C(num_parking),9.0,2.335868e+06,2.595409e+05,16.745888,9.701631e-28
school_duration,1.0,9.008596e+04,9.008596e+04,5.812454,1.592831e-02
school_distance,1.0,1.405483e+04,1.405483e+04,0.906834,3.409754e-01
park_duration,1.0,1.203782e+02,1.203782e+02,0.007767,9.297746e-01
park_distance,1.0,2.742025e+01,2.742025e+01,0.001769,9.664502e-01
shop_duration,1.0,6.268732e+00,6.268732e+00,0.000404,9.839549e-01


In [91]:
mean_absolute_error(lm.predict(X_test).fillna(0), y_test)

140.0890323992437